<a href="https://colab.research.google.com/github/mkielo3/mammalian_brains/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
if 'google.colab' in sys.modules:
	%pip install -q numpy pandas scipy scikit-learn matplotlib seaborn pillow ipykernel torch torchvision torchaudio altair datasets numba numbasom==0.0.5 tensorboard pysal vl-convert-python soundfile

	!git clone https://github.com/mkielo3/mammalian_brains.git
	%cd mammalian_brains

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.2 MB/s eta 0:00:00
 

In [2]:
import torch
import pandas as pd

from tqdm import tqdm
from numbasom.core import lattice_closest_vectors
from analysis.som import SOM
from utils import save_output_to_pickle
from models.olfaction.olfaction import Olfaction
from models.vision.vision import Vision
from models.audio.audio import Audio
from models.touch.touch import Touch
from models.memory.memory import Memory
from utils import save_som_plot, save_rf_plot

from config import Args
args = Args()


fast = True
args.som_size = (5,5) if fast else (25, 25)
args.experiment_name = "main_results_fast" if fast else "main_results"
args.fast = fast
args.som_epochs = 10000 if fast else 100000
modality_list = [Olfaction(args), Vision(args), Audio(args), Touch(args), Memory(args)]

In [3]:
for modality in modality_list:
	print ("\n", modality.modality, pd.Timestamp.now())

	# 1. Train/Download Model
	modality.setup_model()
	modality.setup_som()

	# 2. Get activations for each patch
	patches = modality.get_patches()
	activation_list = []
	for p in tqdm(patches):
		p, static = modality.generate_static(p)
		activation = modality.calculate_activations(static)
		activation_list.append([p, activation])

	# 3. Fit SOM
	x_mat = torch.stack([x[1] for x in activation_list]).numpy()
	som = modality.initialize_som(SOM)
	lattice = som.train(x_mat, num_iterations=args.som_epochs, initialize=args.som_init, normalize=False, start_lrate=args.som_lr)

	# 4. Get coordinates for each BMU
	coordinate_list = [x[0] for x in activation_list]
	closest = lattice_closest_vectors(x_mat, lattice, additional_list=coordinate_list)

	# 5. Save
	output = {"closest": closest,
			"coord_map": coordinate_list,
			"x_range": (0, max([x[0][0] for x in activation_list])),
			"y_range": (0, max([x[0][1] for x in activation_list])),
			"lattice": lattice,
			"som": None,
			"samples": modality.sample_data,
			"modality": modality.modality,
			"args": args,
			"activations": activation_list}

	save_output_to_pickle(output, args.experiment_name)



/content/mammalian_brains/models/olfaction/olfaction.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{self.project_path}/models/olfac


 olfaction 2025-02-12 17:07:52.488785


100%|██████████| 4096/4096 [00:05<00:00, 696.86it/s]


Initializing SOM with Random
Done Init
SOM training took: 3.899162 seconds.
Finding closest data points took: 0.208529 seconds.
Saved output to: output/main_results_fast/olfaction.pkl

 vision 2025-02-12 17:08:02.884584


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 155MB/s]
/content/mammalian_brains/models/vision/vision.py:84: FutureWarning: You are using `torch

...

Loaded "ObjectClusterDataset" - split "test" with 16119 records...
[TouchDataset] Refreshing tuples...
Base LR = 1.000000e-03
tensor(0.5800)


100%|██████████| 548/548 [00:03<00:00, 167.47it/s]


Initializing SOM with Random
Done Init
SOM training took: 3.444031 seconds.
Finding closest data points took: 0.181586 seconds.
Saved output to: output/main_results_fast/touch.pkl

 memory 2025-02-12 17:10:46.784998


100%|██████████| 4455/4455 [00:00<00:00, 77794.53it/s]


Initializing SOM with Random
Done Init
SOM training took: 0.199127 seconds.
Finding closest data points took: 0.081490 seconds.
Saved output to: output/main_results_fast/memory.pkl


In [4]:
save_som_plot(args.experiment_name, modality_list, args)

alt.HConcatChart(...)

In [5]:
save_rf_plot(args.experiment_name, modality_list)

range(0, 5)


alt.HConcatChart(...)